<a href="https://colab.research.google.com/github/ftempesta/Data-Science-Online/blob/master/Laboratorio_4_An%C3%A1lisis_de_Itemsets_frecuentes_y_Reglas_de_Asociaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial

## Preámbulo

Usaremos la librería `mlxtend`, que contiene algunas herramientas adicionales que no se encuentran en `sklearn`.

In [ ]:
!pip install mlxtend

Primero creamos un dataset de transacciones. En este caso, cada transacción corresponde a las compras hechas en un almacén.

In [ ]:
dataset = [['Leche', 'Cebollas', 'Pisco', 'Porotos', 'Huevos', 'Yogurt'],
           ['Eneldo', 'Cebollas', 'Pisco', 'Porotos', 'Huevos', 'Yogurt'],
           ['Leche', 'Manzanas', 'Porotos', 'Huevos'],
           ['Leche', 'Tomillo', 'Espinaca', 'Porotos', 'Yogurt'],
           ['Espinaca', 'Cebollas', 'Cebollas', 'Porotos', 'Helado', 'Huevos']]

Importamos el `TransactionEncoder` que convierte los datos en una matriz de ocurrencias: cada columna corresponde a alguno de los items, cada fila es la misma transacción, y cada celda es 0 o 1 dependiendo de si la transacción contiene al item.

¿A qué se parece este encoder? ¿En qué se diferencia?

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)

df = pd.DataFrame(te_ary, columns=te.columns_)
df

## Itemsets frecuentes con Apriori

Usamos el método `apriori` para generar itemsets frecuentes con un soporte mínimo:

In [ ]:
from mlxtend.frequent_patterns import apriori

apriori(df, min_support=0.6)

Con el parámetro `use_colnames` podemos recuperar los nombres de las columnas en cada itemset:

In [ ]:
apriori(df, min_support=0.6, use_colnames=True)

,support,itemsets
0,0.6,(Cebollas)
1,0.8,(Huevos)
2,0.6,(Leche)
3,1.0,(Porotos)
4,0.6,(Yogurt)
5,0.6,"(Cebollas, Huevos)"
6,0.6,"(Cebollas, Porotos)"
7,0.8,"(Porotos, Huevos)"
8,0.6,"(Porotos, Leche)"
9,0.6,"(Porotos, Yogurt)"


El output de `apriori` es un DataFrame de pandas, por lo que podemos manipularlo igual que a un DataFrame cualquiera.

Por ejemplo, vamos a ordenarlo por soporte y a agregar una nueva columna que indique el tamaño del itemset:

In [ ]:
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.sort_values(by='support', ascending=False, inplace=True)

frequent_itemsets

Podemos hacer filtros más elaborados usando pandas. 

Por ejemplo: ¿cuáles itemsets de largo 2 tienen soporte mayor o igual a 80%?

In [ ]:
frequent_itemsets.loc[
  (frequent_itemsets['length'] == 2) &
  (frequent_itemsets['support'] >= 0.8) 
]

¿Cuáles itemsets corresponden a huevos y cebollas?

In [ ]:
frequent_itemsets.loc[ frequent_itemsets['itemsets'] == {'Cebollas', 'Huevos'} ]

## {a, b, c, ...} es un conjunto, por lo que el orden no importa:
#frequent_itemsets.loc[ frequent_itemsets['itemsets'] == {'Huevos', 'Cebollas'} ]

## y para consultar cuáles itemsets contienen al menos ciertos items:
frequent_itemsets.loc[
  frequent_itemsets['itemsets'].apply(lambda col: col.issuperset({'Huevos', 'Cebollas'}))
]

## Reglas de Asociación

La función `association_rules` permite generar reglas usando un umbral de support, confidence, u otras medidas de interés:

In [ ]:
from mlxtend.frequent_patterns import association_rules

association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

El `antecedents` o `LHS` ("left hand side") es el lado izquierdo de la regla. `consequents` o `RHS` es el lado derecho.

Por ejemplo, la primera regla es $\{\text{Porotos}\} \rightarrow \{\text{Huevos}\}$


Las medidas de interés que muestra la librería son las siguientes

- $\text{support}(X \rightarrow Y) = \text{support}(X \cup Y)$

- $\text{confidence}(X \rightarrow Y) = \frac{\text{support}(X \rightarrow Y)}{\text{support}(X)}$

- $\text{lift}(X \rightarrow Y) = \frac{\text{confidence}(X \rightarrow Y)}{\text{support}(Y)} = \frac{\text{support}(X \rightarrow Y)}{\text{support}(X)  \text{support}(Y)}$

- $\text{leverage}(X \rightarrow Y) = \text{support}(X \rightarrow Y) - \text{support}(X) \times \text{support}(Y)$

- $\text{conviction}(X \rightarrow Y) = \frac{1 - \text{support}(Y)}{1 - \text{confidence}(X \rightarrow Y)}$

Podemos cambiar la medida de interés y aplicar otro filtro para la generación de reglas. Si queremos aplicar más de un filtro, podemos hacerlo posterior a la generación de las reglas.

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules

Para filtrar por el tamaño del LHS, del RHS, o ambos, podemos crear una columna nueva con el tamaño del conjunto y luego hacer el filtro correspondiente

In [ ]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

In [ ]:
rules[ (rules['antecedent_len'] >= 2) &
       (rules['confidence'] > 0.75) &
       (rules['lift'] > 1.2) ]

In [ ]:
rules[rules['antecedents'] == {'Porotos', 'Cebollas'}]

# Laboratorio

Vamos a cargar un dataset que corresponde a las respuestas a un cuestionario hecho a personas en un mall en San Francisco en los años 80.

La descripción completa del dataset se encuentra acá:
https://web.stanford.edu/~hastie/ElemStatLearn/datasets/marketing.info.txt

Cada columna del dataset corresponde a una pregunta en particular, por ejemplo, la primera (columna `income`) corresponde al rango de ingreso del grupo familiar. Los datos son nominales y ordinales (por ejemplo, `income` es ordinal, mientras que `sex` es nominal o categórico).



In [ ]:
column_names = ['income', 'sex', 'age', 'education', 'occupation', 'years-sfo', 'dual-income', 'people', 'children', 'house-status', 'home-type', 'ethnic', 'lang']

survey = pd.read_table('https://web.stanford.edu/~hastie/ElemStatLearn/datasets/marketing.data', 
                       delim_whitespace=True, 
                       header=None,
                       index_col=False,
                       names=column_names)

survey

Vamos a eliminar todas las filas que tengan algún dato faltante y a considerar todas las columnas como enteros (para evitar datos con decimales, aunque todos los valores sean números enteros, esto nos será útil más abajo).

In [ ]:
survey = survey.dropna(axis=0, how='any')
survey = survey.astype('int')
survey

Vamos a hacer one-hot-encoding de los datos de forma de tener valores binarios en las celdas.

Cada fila es una transacción, y cada item será la respuesta específica a cada pregunta, donde se representará como una fila con un `1` en la columna correspondiente a la respuesta a una pregunta.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
survey_cat = enc.fit_transform(survey)


# estos son los nombres de las columnas o items
# cada item es la respuesta específica a una pregunta
enc.get_feature_names(column_names)

El resultado de `enc.fit_transform` es una matriz "esparsa" (sparse matrix), una matriz con muchos 0, que se representa como una estructura de datos distinta a una lista de listas con 0 y 1.

Es posible usar la librería con datos representados como sparse matrix, pero en este caso vamos a "densificar" la matriz convirtiéndola en un DataFrame tal como los hemos conocido hasta ahora.

In [ ]:
survey_sp = pd.DataFrame.sparse.from_spmatrix(survey_cat, columns=enc.get_feature_names(column_names))
survey_sp = survey_sp.astype('category')

survey_sp

Ahora generamos itemset frecuentes con support de al menos 10%:

In [ ]:
frequent_itemsets = apriori(survey_sp, min_support=0.1, use_colnames=True)

frequent_itemsets

Finalmente, generamos reglas. En este caso usaremos un umbral de `confidence` del 70% y agregaremos la columna correspondiente para el largo del antecedente y del consecuente.

In [ ]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequent_len"] = rules["consequents"].apply(lambda x: len(x))

rules

In [ ]:
rules[ (rules['antecedent_len'] >= 2) &
       (rules['confidence'] > 0.75) &
       (rules['lift'] > 1.2) ]

**Pregunta**


Haga una exploración de estos datos.

Modifique el código arriba o escriba nuevo código. Consulte [la especificación de los datos](https://web.stanford.edu/~hastie/ElemStatLearn/datasets/marketing.info.txt). Puede manipular o procesar los datos como estime conveniente. Debe documentar cualquier procesamiento adicional que haga sobre los datos.

Plantee una pregunta "no trivial" (por ejemplo: ¿familias con niños ganan más que familias sin niños?) y haga una exploración de las reglas que permitan responder la pregunta. Incluya el código que usó para sustentar sus conclusiones. 